# How to edit morphologies

`Jaxley` provides functionality to edit morphologies. In particular, it provides functions to delete parts of a morphology or to connect two cell morphologies into a single cell.

> **⚠️ IMPORTANT!**  
> If you edit morphologies, please do so _before_ you change the number of compartments per branch (via, e.g., `cell.branch(0).set_ncomp(4)`). In addition, you must delete all recordings, stimuli, and trainable parameters before running `morph_delete()` or `morph_connect()`.

## Deleting parts of a morphology

Below, we will show how you can delete all apical branches of a morphology.

```python
import jaxley as jx
from jaxley.morphology import morph_delete

cell = jx.read_swc("path_to_swc_file.swc", ncomp=1)

# Creates a new cell which has the apical dendrite deleted.
cell = morph_delete(cell.apical)

# Creates a new cell which has branches 10, 11, 12 deleted.
cell = morph_delete(cell.branch([10, 11, 12])
```

## Attaching two morphologies

`Jaxley` also provides functionality to easily attach two morphologies. This is useful to, for example, replace the axon with a "stub". Below, we show how one can attach two morphologies. We read one morphology from an SWC file and create the other one from scratch.

```python
import jaxley as jx
from jaxley.morphology import morph_connect

# Read a cell from SWC.
cell = jx.read_swc("path_to_swc_file.swc", ncomp=1)

# Create a "stub" of 50um length.
stub = jx.Cell()
stub.set("length", 50.0)

new_cell = morph_connect(cell.branch(0).loc(0.0), stub.branch(0).loc(0.0))
```

Note that you have to use `.loc()` to specify the location at which to connect the cells. The location must be either `0.0` or `1.0`.

## Graph-backend for more flexibility

All of the above is achieved by using `Jaxley`'s "graph-backend", which is based on `networkX`. As such, you can modify `Jaxley` morphologies with any feature of the [networkX](https://networkx.org) toolkit.

For more details on `Jaxley`'s graph-backend, see [this tutorial](https://jaxley.readthedocs.io/en/latest/tutorials/13_graph_backend.html), where we also provide an example of how to trim dendrites of a morphology.